In [3]:
import glob
import pandas as pd

In [4]:
flist = sorted(glob.glob('/scratch/kl02/vhl548/gpm_output/overpass/*.csv'))

In [5]:
offset = pd.read_csv('./gr_offset_2020.csv')

In [16]:
rid = [f[-6:-4] for f in flist]

In [19]:
def get_offset(nid):
    if type(nid) is str:
        nid = int(nid)
    try:
        off = offset.offset[offset.rid == nid].values[0]
    except IndexError:
        off = 0
    return off

In [20]:
def get_qubs(nid, refl_thld, offset):
    txt = f'''#!/bin/bash
#PBS -P kl02
#PBS -q normal
#PBS -l walltime=3:30:00
#PBS -l mem=64GB
#PBS -l wd
#PBS -l jobfs=1GB
#PBS -l ncpus=16
#PBS -lstorage=scratch/kl02+gdata/rq0

source activate radar
python national_archive.py -r {nid} -g {refl_thld} -f {offset} -o /scratch/kl02/vhl548/gpm_output/vmatch/
'''
    return txt

In [22]:
for r in rid:
    off = get_offset(r)
    txt = get_qubs(r, 10, off)
    outfile = f'../scripts/qvmatch_{r}.pbs'
    with open(outfile, 'w+') as fid:
        fid.write(txt)